# Basic VOI Analysis for a New Drug

This notebook demonstrates a basic Value of Information (VOI) analysis for a new drug compared to the current standard of care. We will calculate the Expected Value of Perfect Information (EVPI) to determine the maximum value of reducing all uncertainty in the model.

## 1. Define the Model

We will use a simple model with two parameters: the cost of the new drug and its effectiveness (in QALYs).

In [ ]:
import numpy as np
from voiage import DecisionAnalysis, ParameterSet, ValueArray

# Define the model parameters
parameters = {
    "cost_new_drug": np.random.normal(loc=10000, scale=1000, size=1000),
    "effectiveness_new_drug": np.random.normal(loc=2, scale=0.2, size=1000),
}
psa = ParameterSet(parameters=parameters)

# Define the model function
def model_func(psa: ParameterSet):
    wtp = 20000
    cost_standard_care = 5000
    effectiveness_standard_care = 1.5

    nb_new_drug = psa.parameters["effectiveness_new_drug"] * wtp - psa.parameters["cost_new_drug"]
    nb_standard_care = effectiveness_standard_care * wtp - cost_standard_care

    return ValueArray(
        values=np.stack([nb_new_drug, np.full_like(nb_new_drug, nb_standard_care)], axis=1),
        strategy_names=["New Drug", "Standard of Care"],
    )

# Create a DecisionAnalysis object
analysis = DecisionAnalysis(parameters=psa, values=model_func(psa))

## 2. Calculate EVPI

In [ ]:
evpi = analysis.evpi()
print(f"EVPI: ${evpi:,.2f}")

## 3. Plot the Cost-Effectiveness Plane

In [ ]:
analysis.plot_ce_plane(wtp=20000)

## 4. Plot the Cost-Effectiveness Acceptability Curve (CEAC)

In [ ]:
analysis.plot_ceac(wtp_range=np.linspace(0, 40000, 100))

## 5. Plot the VOI Curves

In [ ]:
analysis.plot_voi_curves(wtp_range=np.linspace(0, 40000, 100))